<a href="https://colab.research.google.com/github/hey-ABHAY/Deep_Learning/blob/main/NLP_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

data=gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence  import pad_sequences
from sklearn.model_selection import train_test_split

with open('hamlet.txt','r') as file:
    text=file.read().lower()
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words


4818

In [ ]:
#create imput sequences
input_sequence=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range (1,len(token_list)):
        n_gram_Sequence=token_list[:i+1]
        input_sequence.append(n_gram_Sequence)

In [ ]:
#pad sequences
max_sequence_len=max([len(x) for x in input_sequence])
max_sequence_len
#it means that the largets sequence is of length 14

14

In [ ]:
input_sequence=np.array(pad_sequences(input_sequence,maxlen=max_sequence_len,padding='pre'))
input_sequence

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [ ]:
x,y=input_sequence[:,:-1],input_sequence[:,-1]

y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [ ]:
#split
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2)


In [ ]:
# train the lstm model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout

#define the model
model=Sequential()

model.add(Embedding(total_words,100,input_length=max_sequence_len))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

#compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None,max_sequence_len))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 14, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(20585, 13) (20585, 4818)
(5147, 13) (5147, 4818)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)


In [ ]:
#Train the model
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1,callbacks=[earlystopping])


Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 47s 67ms/step - accuracy: 0.0279 - loss: 7.1360 - val_accuracy: 0.0293 - val_loss: 6.7499
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 79s 63ms/step - accuracy: 0.0356 - loss: 6.4698 - val_accuracy: 0.0455 - val_loss: 6.8381
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 39s 60ms/step - accuracy: 0.0423 - loss: 6.2998 - val_accuracy: 0.0525 - val_loss: 6.9183
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 60ms/step - accuracy: 0.0476 - loss: 6.2018 - val_accuracy: 0.0525 - val_loss: 6.9038
